In [1]:
import overpy
import pandas as pd

In [2]:
api = overpy.Overpass()
result = api.query("""
    nwr[building=church](45.2440, 10.1321, 46.0932, 11.9861);
    out center;
    """)
result

In [3]:
"Found {} churches ({} nodes, {} ways, {} relations)!".format(len(result.nodes)+len(result.ways)+len(result.relations),
                                                                    len(result.nodes),
                                                                    len(result.ways),
                                                                    len(result.relations))

'Found 5440 churches (12 nodes, 5387 ways, 41 relations)!'

In [4]:
names, lats, lons = [], [], []

for node in result.nodes:
    if "name" in node.tags:
        names.append(node.tags["name"])
        lats.append(node.lat)
        lons.append(node.lon)

for way in result.ways:
    if "name" in way.tags:
        names.append(way.tags["name"])
        lats.append(way.center_lat)
        lons.append(way.center_lon)

for relation in result.relations:
    if "name" in relation.tags:
        names.append(relation.tags["name"])
        lats.append(relation.center_lat)
        lons.append(relation.center_lon)

churches = pd.DataFrame(zip(names,lats,lons), columns=["name","lat","lon"])
churches

,name,lat,lon
0,Chiesetta San Valentino,45.6385048,11.3977294
1,Chiesetta di Restele,45.8781317,11.3045914
2,Maria Ausiliatrice,45.8879163,11.0447089
3,Chiesetta del Carmine,45.6248031,11.3948993
4,Chiesa Gentilata,45.6612328,11.3748491
...,...,...,...
1562,Chiesa di San Zenone ed Eurosia,45.5974236,10.1366088
1563,Chiesa di Santa Maria Assunta,45.5277409,10.1835815
1564,Chiesa e convento di San Pancrazio,45.4015205,11.5657326
1565,Chiesa di Sant'Andrea,45.9514940,10.2705945


In [5]:
import reverse_geocoder as rg

churches['town'] = churches.apply(lambda r: rg.search((r["lat"],r["lon"]))[0]["name"], axis=1)
churches

Loading formatted geocoded file...


,name,lat,lon,town
0,Chiesetta San Valentino,45.6385048,11.3977294,Malo
1,Chiesetta di Restele,45.8781317,11.3045914,Longhi
2,Maria Ausiliatrice,45.8879163,11.0447089,Rovereto
3,Chiesetta del Carmine,45.6248031,11.3948993,Cereda-Cozza Cornedo
4,Chiesa Gentilata,45.6612328,11.3748491,Monte di Malo
...,...,...,...,...
1562,Chiesa di San Zenone ed Eurosia,45.5974236,10.1366088,Ronco
1563,Chiesa di Santa Maria Assunta,45.5277409,10.1835815,Roncadelle
1564,Chiesa e convento di San Pancrazio,45.4015205,11.5657326,Ponte di Barbarano
1565,Chiesa di Sant'Andrea,45.9514940,10.2705945,Malegno


In [21]:
import unidecode

churches["name"] = churches.apply(lambda r: unidecode.unidecode(r["name"]), axis=1)
churches["town"] = churches.apply(lambda r: unidecode.unidecode(r["town"]), axis=1)
churches

,name,lat,lon,town
0,Chiesetta San Valentino,45.6385048,11.3977294,Malo
1,Chiesetta di Restele,45.8781317,11.3045914,Longhi
2,Maria Ausiliatrice,45.8879163,11.0447089,Rovereto
3,Chiesetta del Carmine,45.6248031,11.3948993,Cereda-Cozza Cornedo
4,Chiesa Gentilata,45.6612328,11.3748491,Monte di Malo
...,...,...,...,...
1562,Chiesa di San Zenone ed Eurosia,45.5974236,10.1366088,Ronco
1563,Chiesa di Santa Maria Assunta,45.5277409,10.1835815,Roncadelle
1564,Chiesa e convento di San Pancrazio,45.4015205,11.5657326,Ponte di Barbarano
1565,Chiesa di Sant'Andrea,45.9514940,10.2705945,Malegno


In [22]:
churches.to_csv("churches.csv", index=False)

In [23]:
from google_images_download import google_images_download

search_queries = churches["town"] + " " + churches["name"]

for q in search_queries:
    response = google_images_download.googleimagesdownload()
    paths = response.download({"keywords": "{}".format(q),
                               "limit": 6})


Item no.: 1 --> Item name = Malo Chiesetta San Valentino
Evaluating...
Starting Download...
Completed Image ====> 1.52510843.jpg
Completed Image ====> 2.PANORAMA-DALLA-CHIESETTA-SAN-VALENTINO-A-MALO-VICENZA.jpg
Completed Image ====> 3.23378968.jpg
Completed Image ====> 4.B_622f5a2a0b04645186d2f0d402a9add7.jpg
URLError on an image...trying next one... Error: HTTP Error 503: Service Temporarily Unavailable
Completed Image ====> 5.chiesa-di-san-valentino.jpg
URLError on an image...trying next one... Error: HTTP Error 503: Service Temporarily Unavailable
Completed Image ====> 6.SANTUARIO-SANTA-LIBERA-A-MALOjpg-1.jpg

Errors: 2


Item no.: 1 --> Item name = Longhi Chiesetta di Restele
Evaluating...
Starting Download...
Completed Image ====> 1.Cappella_arcivescovile_Ravenna_3.JPG
Completed Image ====> 2.440px-Santa_Maria_in_Aracoeli%3B_Cappella_Bufalini%3B_Pinturicchio_1.jpg
Completed Image ====> 3.Cappella_arcivescovile_Ravenna_1.JPG
Completed Image ====> 4.SMiA_Treppe_und_Fassade.JPG
Comp